### Assistente para Análise de Dados

In [1]:
import openai
import pandas as pd

In [3]:
client = openai.Client()

In [ ]:
# Enviar arquivo CSV para OpenAI
file = client.files.create(
    file = open("files/sales_data.csv", "rb"),
    purpose = "assistants")

In [5]:
print(file.id)

file-P6GMTHyTM866xGtPknesHu


In [6]:
assistant = client.beta.assistants.create(
    name = "Analista de Dados",
    instructions = "Você é um anasta que analisa dados sobre vendas",
    tools = [{"type":"code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}},
    model="gpt-4o")

In [16]:
#1) pergunta = "Qual o rating médio das vendas do supermercado?"
#2) pergunta
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto."

In [17]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta)

C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\3798620811.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\3798620811.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [18]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome de usuário premium")

C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\3617845786.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [19]:
# Aguarda o thread rodar
import time 
while run.status in ["queued","in_progress","cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id)

C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\953612562.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


In [20]:
# Verificar a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id = thread.id
    )

    print(mensagens)
else:
    print(f"Erro: {run.status}")

C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\1930001157.py:3: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_WgLLih2XLqDUMGJRkNqn18PW', assistant_id='asst_KcN98YwaaKiIyoVaVlI6uILT', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-DnyJ9R6GsmntJHGYNv7fWJ', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza mostrando o percentual de vendas por linha de produto. Cada fatia representa a contribuição percentual de uma linha de produto específica para o total de vendas. Se precisar de mais alguma coisa, fique à vontade para perguntar!'), type='text')], created_at=1756036560, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_BSBTmfCNM7g2fdDdPcdmP5W5', status=None, thread_id='thread_kCqY4CQlXsnPJGbZuodp1mVJ'), Message(id='msg_enx9zapH9jrnnxO0KcKY9opX', assistant_id='asst_KcN98YwaaKiIyoVaVlI6uILT', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[]

In [ ]:
#mensagens.data[0].content[0].text.value

'O rating médio das vendas do supermercado é aproximadamente 6,89.'

In [21]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id)

C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\764390232.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [24]:
for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("=" *10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id = thread.id,
            message_id = step.step_details.message_creation.message_id)

        # Tratamento diferenciado dependendo do tipo da resposta do Assistant (texto, arquivo de imagem etc.)
        #print(message.content[0].text.value)
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem: {file.id} foi salva.")
        
        



===Step message_creation


C:\Users\Naja\AppData\Local\Temp\ipykernel_6392\414113233.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Para gerar um gráfico de pizza que mostre o percentual de vendas por linha de produto, precisamos primeiro examinar o conteúdo do arquivo enviado para identificar a estrutura dos dados. Vamos dar uma olhada nas primeiras linhas do arquivo para entender como os dados estão organizados.

===Step tool_calls
# Vamos começar lendo as primeiras linhas do arquivo para entender sua estrutura.
import pandas as pd

# Carregar o arquivo
file_path = '/mnt/data/file-P6GMTHyTM866xGtPknesHu'
data = pd.read_csv(file_path)

# Mostrar as primeiras linhas do DataFrame
data.head()

===Step message_creation
O arquivo contém dados de vendas, incluindo vários campos relevantes. Para gerar o gráfico de pizza desejado, vamos focar nas colunas `Product line` e `Total`, onde cada linha de produto precisa ser somada em termos de vendas totais para calcular o percentual de contribuição de cada linha de produto.

Vamos calcular as vendas totais de cada linha de produto e, em seguida, iremos plotar o gráfico de pizz